In [26]:
filepath = "test_data/H2.wfn"

include("./wfn.jl");
include("./ext21.jl");
#TODO: Use precompilation

In [27]:
#Struct containing all relevant WFN information
f = read_wfn(filepath, device = cpu);
f.title

"H2_gas"

In [28]:
f2 = read_wfx("test_data/H2.wfx", device = cpu);
f2.title

"optimization"

In [29]:
f.mo

14×18 Matrix{Float32}:
  0.0482668     0.0870664     0.128359     …   2.83354f-17   0.020738
  0.108083      0.194966      0.287431        -4.44461f-16   0.0560565
 -0.112172     -0.202343     -0.298307        -1.94244f-14  -0.241326
 -7.78452f-17  -1.40422f-16  -2.07018f-16      1.26811f-18  -2.53968f-16
 -6.43044f-15  -1.15996f-14  -1.71008f-14      0.577846      1.14848f-14
  0.253624      0.457501      0.674477     …  -6.59412f-16  -0.300979
  0.173712      0.313352      0.461964         7.83366f-15  -0.667466
 -5.39478f-16  -9.73142f-16  -1.43467f-15     -0.919882      1.32992f-15
  1.9773f-16    3.56676f-16   5.25835f-16      0.330736     -3.88247f-16
  0.212193      0.382766      0.564298        -2.22556f-15   0.124451
  0.149458      0.269601      0.397462     …   1.81513f-15  -2.17979
 -0.0823477    -0.148544     -0.218992         4.90503f-16  -0.920859
  0.0478553     0.0863242     0.127265        -7.27137f-16   0.0389374
 -0.0163285    -0.0294543    -0.0434233       -9.43572

In [30]:
f.n_primitives

18

In [31]:
#Points to evaluate
r⃗ = [0 0 0;
     0.1 0 0.7;
     0 0 -0.7;
     0 1 0;
    1 -1 0;
    0.4 0.5 0.5;
    1 1 1;
    0 0 0.6;
    0 0 0.5;
    0 10 10] .|> Float32 |> cpu


10×3 Matrix{Float32}:
 0.0   0.0   0.0
 0.1   0.0   0.7
 0.0   0.0  -0.7
 0.0   1.0   0.0
 1.0  -1.0   0.0
 0.4   0.5   0.5
 1.0   1.0   1.0
 0.0   0.0   0.6
 0.0   0.0   0.5
 0.0  10.0  10.0

### Laplacian of electronic density

In [37]:
get_electronic_density_laplacian(r⃗,f2)

10-element Vector{Float64}:
  -1.0137217054324417
 -15.352159366450865
 -25.340914891713506
   0.057310950167120546
   0.07405967571824977
  -0.1402936032508163
   0.05404269279051238
 -14.731390777342668
  -4.716604672183271
   5.557325309450305e-19

### Broyden

In [21]:
#Broyden version. Numerically unstable
#Results are different between CPU and GPU
function get_change_gradient(dim,∂X,∂Y,∂Z,∂X_prev,∂Y_prev,∂Z_prev)::Float32
    dim == 1 ? ∂X-∂X_prev :
    dim == 2 ? ∂Y-∂Y_prev :
    ∂Z-∂Z_prev
end
#Code for calculating the critical points of ρ via Broyden's method.
# Work in progress
#function find_critical_ρ_points(r⃗::AbstractVector, f::WFN; iters = 15, conv_check= 1e-7)
    iters = 20
    #save copy of r⃗
    r⃗_prev = copy(r⃗)
    #Assignation of center per primitive
    @tullio r⃗_μ[prim,dim] := f.nuclei_pos[f.center_assignments[prim],dim] grad=false
    #Difference between each proposed point and each nuclei center assigned to a MO
    @tullio Δr⃗[p,dim,r] := r⃗[r,dim] - r⃗_μ[p,dim] grad=false
    #Squared distances
    sq_dist = dropdims(sum(Δr⃗.^2, dims=2), dims=2)
    #Gaussian constant
    @tullio c_g[p,r] := get_gaussian_constant(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    
    @tullio ∂gc∂X[p,r] := get_∂gc∂X(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂gc∂Y[p,r] := get_∂gc∂Y(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂gc∂Z[p,r] := get_∂gc∂Z(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false

    @tullio ∂²gc∂X²[p,r] := get_∂²gc∂X²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂²gc∂XY[p,r] := get_∂²gc∂XY(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂²gc∂XZ[p,r] := get_∂²gc∂XZ(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂²gc∂Y²[p,r] := get_∂²gc∂Y²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂²gc∂YZ[p,r] := get_∂²gc∂YZ(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    @tullio ∂²gc∂Z²[p,r] := get_∂²gc∂Z²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
    
    F0 = exp.(-f.exponents .* sq_dist)
    toalp = -2.0 * f.exponents
    toalpe = toalp .* F0
    ∂F∂X = toalpe .* Δr⃗[:,1,:]
    ∂F∂Y = toalpe .* Δr⃗[:,2,:]
    ∂F∂Z = toalpe .* Δr⃗[:,3,:]
    ∂²F∂X² = (toalp .* Δr⃗[:,1,:] .* ∂F∂X) + toalpe
    ∂²F∂XY = toalp .* Δr⃗[:,2,:] .* ∂F∂X
    ∂²F∂XZ = toalp .* Δr⃗[:,3,:] .* ∂F∂X
    ∂²F∂Y² = (toalp .* Δr⃗[:,2,:] .* ∂F∂Y) + toalpe
    ∂²F∂YZ = toalp .* Δr⃗[:,3,:] .* ∂F∂Y
    ∂²F∂Z² = (toalp .* Δr⃗[:,3,:] .* ∂F∂Z) + toalpe
    
    Ψ_μ = c_g .* F0
    ∂Ψ_μ∂X = ∂gc∂X .* F0 + c_g .* ∂F∂X
    ∂Ψ_μ∂Y = ∂gc∂Y .* F0 + c_g .* ∂F∂Y
    ∂Ψ_μ∂Z = ∂gc∂Z .* F0 + c_g .* ∂F∂Z
    ∂²Ψ_μ∂X² = (∂²gc∂X² .* F0) + (2 * ∂gc∂X .* ∂F∂X) + (c_g .* ∂²F∂X²)
    ∂²Ψ_μ∂XY = (∂²gc∂XY .* F0) + (∂gc∂X .* ∂F∂Y) + (∂gc∂Y .* ∂F∂X) + (c_g .* ∂²F∂XY)
    ∂²Ψ_μ∂XZ = (∂²gc∂XZ .* F0) + (∂gc∂X .* ∂F∂Z) + (∂gc∂Z .* ∂F∂X) + (c_g .* ∂²F∂XZ)
    ∂²Ψ_μ∂Y² = (∂²gc∂Y² .* F0) + (2 * ∂gc∂Y .* ∂F∂Y) + (c_g .* ∂²F∂Y²)
    ∂²Ψ_μ∂YZ = (∂²gc∂YZ .* F0) + (∂gc∂Y .* ∂F∂Z) + (∂gc∂Z .* ∂F∂Y) + (c_g .* ∂²F∂YZ)
    ∂²Ψ_μ∂Z² = (∂²gc∂Z² .* F0) + (2 * ∂gc∂Z .* ∂F∂Z) + (c_g .* ∂²F∂Z²)

    Φ_r = f.mo * Ψ_μ
    ∂Φ∂X = f.mo * ∂Ψ_μ∂X
    ∂Φ∂Y = f.mo * ∂Ψ_μ∂Y
    ∂Φ∂Z = f.mo * ∂Ψ_μ∂Z
    ∂²Φ∂X² = f.mo * ∂²Ψ_μ∂X²
    ∂²Φ∂XY = f.mo * ∂²Ψ_μ∂XY
    ∂²Φ∂XZ = f.mo * ∂²Ψ_μ∂XZ
    ∂²Φ∂Y² = f.mo * ∂²Ψ_μ∂Y²
    ∂²Φ∂YZ = f.mo * ∂²Ψ_μ∂YZ
    ∂²Φ∂Z² = f.mo * ∂²Ψ_μ∂Z²

    #ρ = electronic density for proposed points
    ρ = transpose(Φ_r.^2) * f.occ_no
    #Newton/pseudo-Newton methods optimize 3 equations for 3 variables:
    #Variables: X, Y, Z of each point
    #Equations: Gradient of ρ at X, Y, Z
    ∂ρ∂X = transpose(∂Φ∂X .* Φ_r) * f.occ_no
    ∂ρ∂Y = transpose(∂Φ∂Y .* Φ_r) * f.occ_no
    ∂ρ∂Z = transpose(∂Φ∂Z .* Φ_r) * f.occ_no
    #Newton/pseudo-Newton method need the inverse jacobian of the function to optimize.
    #Calculating second derivatives of ρ to obtain the Hessian
    ∂²ρ∂X² = 2 * (transpose(∂Φ∂X.^2)  + transpose(∂²Φ∂X² .* Φ_r)) * f.occ_no
    ∂²ρ∂XY = 2 * (transpose(∂Φ∂X .* ∂Φ∂Y)  + transpose(∂²Φ∂XY .* Φ_r)) * f.occ_no
    ∂²ρ∂XZ = 2 * (transpose(∂Φ∂X .* ∂Φ∂Z)  + transpose(∂²Φ∂XZ .* Φ_r)) * f.occ_no
    ∂²ρ∂Y² = 2 * (transpose(∂Φ∂Y.^2) + transpose(∂²Φ∂Y² .* Φ_r)) * f.occ_no
    ∂²ρ∂YZ = 2 * (transpose(∂Φ∂Y .* ∂Φ∂Z)  + transpose(∂²Φ∂YZ .* Φ_r)) * f.occ_no
    ∂²ρ∂Z² = 2 * (transpose(∂Φ∂Z.^2) + transpose(∂²Φ∂Z² .* Φ_r)) * f.occ_no

    #Generating the inverse of the Hessian of ρ by determinants/cofactors
    #Evaluating as a single expression because of speed.
    #Otherwise launches multiple kernels, which is slower than simply repeating the operation
    @tullio inv_H[m,n,p] := generate_Y_matrix_el(m, n, ∂²ρ∂X²[p],
        ∂²ρ∂XY[p],
        ∂²ρ∂XZ[p],
        ∂²ρ∂Y²[p],
        ∂²ρ∂YZ[p],
        ∂²ρ∂Z²[p]) / (∂²ρ∂X²[p] * generate_Y_matrix_el(1, 1, ∂²ρ∂X²[p],
        ∂²ρ∂XY[p],
        ∂²ρ∂XZ[p],
        ∂²ρ∂Y²[p],
        ∂²ρ∂YZ[p],
        ∂²ρ∂Z²[p]) + ∂²ρ∂XY[p] * generate_Y_matrix_el(2, 1, ∂²ρ∂X²[p],
        ∂²ρ∂XY[p],
        ∂²ρ∂XZ[p],
        ∂²ρ∂Y²[p],
        ∂²ρ∂YZ[p],
        ∂²ρ∂Z²[p]) + ∂²ρ∂XZ[p] * generate_Y_matrix_el(3, 1, ∂²ρ∂X²[p],
        ∂²ρ∂XY[p],
        ∂²ρ∂XZ[p],
        ∂²ρ∂Y²[p],
        ∂²ρ∂YZ[p],
        ∂²ρ∂Z²[p])) (m in 1:3, n in 1:3) grad=false

    
    #Try and update inv_H, if iter != 1
    for iter in 1:iters
        if iter != 1
            #Save last gradients
            ∂ρ∂X_prev = copy(∂ρ∂X)
            ∂ρ∂Y_prev = copy(∂ρ∂Y)
            ∂ρ∂Z_prev = copy(∂ρ∂Z)
            #Calculate new gradients
            @tullio Δr⃗[p,dim,r] = r⃗[r,dim] - r⃗_μ[p,dim] grad=false
            #Squared distances
            sq_dist = dropdims(sum(Δr⃗.^2, dims=2), dims=2)
            #Gaussian constant
            @tullio c_g[p,r] = get_gaussian_constant(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
            @tullio ∂gc∂X[p,r] = get_∂gc∂X(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
            @tullio ∂gc∂Y[p,r] = get_∂gc∂Y(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
            @tullio ∂gc∂Z[p,r] = get_∂gc∂Z(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false

            ∂F∂X = -2.0 * f.exponents .* exp.(-f.exponents .* sq_dist) .* Δr⃗[:,1,:]
            ∂F∂Y = -2.0 * f.exponents .* exp.(-f.exponents .* sq_dist) .* Δr⃗[:,2,:]
            ∂F∂Z = -2.0 * f.exponents .* exp.(-f.exponents .* sq_dist) .* Δr⃗[:,3,:]

            Ψ_μ = c_g .* exp.(-f.exponents .* sq_dist)
            ∂Ψ_μ∂X = ∂gc∂X .* exp.(-f.exponents .* sq_dist) + c_g .* ∂F∂X
            ∂Ψ_μ∂Y = ∂gc∂Y .* exp.(-f.exponents .* sq_dist) + c_g .* ∂F∂Y
            ∂Ψ_μ∂Z = ∂gc∂Z .* exp.(-f.exponents .* sq_dist) + c_g .* ∂F∂Z

            Φ_r = f.mo * Ψ_μ
            ∂Φ∂X = f.mo * ∂Ψ_μ∂X
            ∂Φ∂Y = f.mo * ∂Ψ_μ∂Y
            ∂Φ∂Z = f.mo * ∂Ψ_μ∂Z

            ∂ρ∂X = transpose(∂Φ∂X .* Φ_r) * f.occ_no
            ∂ρ∂Y = transpose(∂Φ∂Y .* Φ_r) * f.occ_no
            ∂ρ∂Z = transpose(∂Φ∂Z .* Φ_r) * f.occ_no

            #Calculate change in proposed points, output gradients
            Δprop = r⃗-r⃗_prev
            @tullio Δ∂ρ[p,dim] := get_change_gradient(dim,
                                                      ∂ρ∂X[p],
                                                      ∂ρ∂Y[p],
                                                      ∂ρ∂Z[p],
                                                      ∂ρ∂X_prev[p],
                                                      ∂ρ∂Y_prev[p],
                                                      ∂ρ∂Z_prev[p]) (dim in 1:3) grad=false
            #Update inverse hessian
            @tullio constant[p] := Δprop[p,j] * inv_H[j,i,p] * Δ∂ρ[p,i] grad=false
            #Broyden update step as a tullio expression:
            #For a point p, let:
            #A = Δprop' - inv_H*(Δ∂ρ')
            #M = A*Δprop
            #M*inv_H/const is the Broyden update step, so:
            #inv_H[m,n] += inv_H[:,n]⋅M[m,:]
            #M[m,:] = A[m]*Δprop
            #So:
            #inv_H[m,n] += A[m]*(inv_H[:,n]⋅Δprop)
            # A[m] = Δprop[m] - (inv_H[m,:] ⋅ Δ∂ρ)
            #So the final expression is:
            #inv_H[m,n] += (Δprop[m] - (inv_H[m,:] ⋅ Δ∂ρ))*(inv_H[:,n]⋅Δprop)/constant
            @tullio inv_H[m,n,p] += (inv_H[1,n,p]*Δprop[p,1] + inv_H[2,n,p]*Δprop[p,2] + inv_H[3,n,p]*Δprop[p,3]) * (Δprop[p,m] - (inv_H[m,1,p]*Δ∂ρ[p,1]+inv_H[m,2,p]*Δ∂ρ[p,2]+inv_H[m,3,p]*Δ∂ρ[p,3])) / constant[p] grad = false
        end
        r⃗_prev = copy(r⃗)
        @tullio r⃗[p,dim] += -inv_H[dim,1,p] * ∂ρ∂X[p] - inv_H[dim,2,p] * ∂ρ∂Y[p] - inv_H[dim,3,p] * ∂ρ∂Z[p]
    end
    #Update points


In [22]:
r⃗

10×3 Matrix{Float32}:
 -1.10328f-17   1.11544f-16   2.3029f-13
 -7.73465f-5    7.04789f-18   0.670787
  9.64613f-19   1.21091f-17  -0.670376
  5.20869f-18   2.30317       5.48578f-16
  1.74753      -1.90587      -1.72873f-16
  1.14744       1.54877      -0.206347
  1.65457       1.76259       1.4466
 -1.66219f-18   6.95621f-18   0.670376
 -3.75224f-18   7.25564f-17   0.00158873
  0.0          10.9305       11.1636

In [17]:
#Points to evaluate
r⃗ = [0 0 0;
     0.1 0 0.7;
     0 0 -0.7;
     0 1 0;
    1 -1 0;
    0.4 0.5 0.5;
    1 1 1;
    0 0 0.6;
    0 0 0.5;
    0 10 10] .|> Float32 |> cpu

10×3 Matrix{Float32}:
 0.0   0.0   0.0
 0.1   0.0   0.7
 0.0   0.0  -0.7
 0.0   1.0   0.0
 1.0  -1.0   0.0
 0.4   0.5   0.5
 1.0   1.0   1.0
 0.0   0.0   0.6
 0.0   0.0   0.5
 0.0  10.0  10.0

### Newton-Raphson

In [18]:
#Newton-Raphson method
# Work in progress
#TODO: 
    #Check what kind of critical point was found
    #Tell the user which points converged, which diverged
function find_critical_ρ_points(r⃗, f::WFN; iters = 15, conv_check= 1e-7)
    #To allow us to acces Φ_r outside the loop's scope
    Φ_r = NaN
    for i in 1:iters
    #Assignation of center per primitive
        @tullio r⃗_μ[prim,dim] := f.nuclei_pos[f.center_assignments[prim],dim] grad=false
        #Difference between each proposed point and each nuclei center assigned to a MO
        @tullio Δr⃗[p,dim,r] := r⃗[r,dim] - r⃗_μ[p,dim] grad=false
        #Squared distances
        sq_dist = dropdims(sum(Δr⃗.^2, dims=2), dims=2)
        #Gaussian constant
        @tullio c_g[p,r] := get_gaussian_constant(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false

        @tullio ∂gc∂X[p,r] := get_∂gc∂X(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂gc∂Y[p,r] := get_∂gc∂Y(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂gc∂Z[p,r] := get_∂gc∂Z(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false

        @tullio ∂²gc∂X²[p,r] := get_∂²gc∂X²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂²gc∂XY[p,r] := get_∂²gc∂XY(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂²gc∂XZ[p,r] := get_∂²gc∂XZ(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂²gc∂Y²[p,r] := get_∂²gc∂Y²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂²gc∂YZ[p,r] := get_∂²gc∂YZ(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false
        @tullio ∂²gc∂Z²[p,r] := get_∂²gc∂Z²(f.type_assignments[p], Δr⃗[p,1,r], Δr⃗[p,2,r], Δr⃗[p,3,r]) grad=false

        F0 = exp.(-f.exponents .* sq_dist)
        toalp = -2.0 * f.exponents
        toalpe = toalp .* F0
        ∂F∂X = toalpe .* Δr⃗[:,1,:]
        ∂F∂Y = toalpe .* Δr⃗[:,2,:]
        ∂F∂Z = toalpe .* Δr⃗[:,3,:]
        ∂²F∂X² = (toalp .* Δr⃗[:,1,:] .* ∂F∂X) + toalpe
        ∂²F∂XY = toalp .* Δr⃗[:,2,:] .* ∂F∂X
        ∂²F∂XZ = toalp .* Δr⃗[:,3,:] .* ∂F∂X
        ∂²F∂Y² = (toalp .* Δr⃗[:,2,:] .* ∂F∂Y) + toalpe
        ∂²F∂YZ = toalp .* Δr⃗[:,3,:] .* ∂F∂Y
        ∂²F∂Z² = (toalp .* Δr⃗[:,3,:] .* ∂F∂Z) + toalpe

        Ψ_μ = c_g .* F0
        ∂Ψ_μ∂X = ∂gc∂X .* F0 + c_g .* ∂F∂X
        ∂Ψ_μ∂Y = ∂gc∂Y .* F0 + c_g .* ∂F∂Y
        ∂Ψ_μ∂Z = ∂gc∂Z .* F0 + c_g .* ∂F∂Z
        ∂²Ψ_μ∂X² = (∂²gc∂X² .* F0) + (2 * ∂gc∂X .* ∂F∂X) + (c_g .* ∂²F∂X²)
        ∂²Ψ_μ∂XY = (∂²gc∂XY .* F0) + (∂gc∂X .* ∂F∂Y) + (∂gc∂Y .* ∂F∂X) + (c_g .* ∂²F∂XY)
        ∂²Ψ_μ∂XZ = (∂²gc∂XZ .* F0) + (∂gc∂X .* ∂F∂Z) + (∂gc∂Z .* ∂F∂X) + (c_g .* ∂²F∂XZ)
        ∂²Ψ_μ∂Y² = (∂²gc∂Y² .* F0) + (2 * ∂gc∂Y .* ∂F∂Y) + (c_g .* ∂²F∂Y²)
        ∂²Ψ_μ∂YZ = (∂²gc∂YZ .* F0) + (∂gc∂Y .* ∂F∂Z) + (∂gc∂Z .* ∂F∂Y) + (c_g .* ∂²F∂YZ)
        ∂²Ψ_μ∂Z² = (∂²gc∂Z² .* F0) + (2 * ∂gc∂Z .* ∂F∂Z) + (c_g .* ∂²F∂Z²)

        Φ_r = f.mo * Ψ_μ
        ∂Φ∂X = f.mo * ∂Ψ_μ∂X
        ∂Φ∂Y = f.mo * ∂Ψ_μ∂Y
        ∂Φ∂Z = f.mo * ∂Ψ_μ∂Z
        ∂²Φ∂X² = f.mo * ∂²Ψ_μ∂X²
        ∂²Φ∂XY = f.mo * ∂²Ψ_μ∂XY
        ∂²Φ∂XZ = f.mo * ∂²Ψ_μ∂XZ
        ∂²Φ∂Y² = f.mo * ∂²Ψ_μ∂Y²
        ∂²Φ∂YZ = f.mo * ∂²Ψ_μ∂YZ
        ∂²Φ∂Z² = f.mo * ∂²Ψ_μ∂Z²

        #ρ = electronic density for proposed points
        #ρ = transpose(Φ_r.^2) * f.occ_no
        
        #Newton/pseudo-Newton methods optimize 3 equations for 3 variables:
        #Variables: X, Y, Z of each point
        #Equations: Gradient of ρ at X, Y, Z
        ∂ρ∂X = transpose(∂Φ∂X .* Φ_r) * f.occ_no
        ∂ρ∂Y = transpose(∂Φ∂Y .* Φ_r) * f.occ_no
        ∂ρ∂Z = transpose(∂Φ∂Z .* Φ_r) * f.occ_no
        #Newton/pseudo-Newton method need the inverse jacobian of the function to optimize.
        #Calculating second derivatives of ρ to obtain the Hessian
        ∂²ρ∂X² = 2 * (transpose(∂Φ∂X.^2)  + transpose(∂²Φ∂X² .* Φ_r)) * f.occ_no
        ∂²ρ∂XY = 2 * (transpose(∂Φ∂X .* ∂Φ∂Y)  + transpose(∂²Φ∂XY .* Φ_r)) * f.occ_no
        ∂²ρ∂XZ = 2 * (transpose(∂Φ∂X .* ∂Φ∂Z)  + transpose(∂²Φ∂XZ .* Φ_r)) * f.occ_no
        ∂²ρ∂Y² = 2 * (transpose(∂Φ∂Y.^2) + transpose(∂²Φ∂Y² .* Φ_r)) * f.occ_no
        ∂²ρ∂YZ = 2 * (transpose(∂Φ∂Y .* ∂Φ∂Z)  + transpose(∂²Φ∂YZ .* Φ_r)) * f.occ_no
        ∂²ρ∂Z² = 2 * (transpose(∂Φ∂Z.^2) + transpose(∂²Φ∂Z² .* Φ_r)) * f.occ_no

        #Generating the inverse of the Hessian of ρ by determinants/cofactors
        #Evaluating as a single expression because of speed.
        #Otherwise launches multiple kernels, which is slower than simply repeating the operation
        @tullio inv_H[m,n,p] := generate_Y_matrix_el(m, n, ∂²ρ∂X²[p],
            ∂²ρ∂XY[p],
            ∂²ρ∂XZ[p],
            ∂²ρ∂Y²[p],
            ∂²ρ∂YZ[p],
            ∂²ρ∂Z²[p]) / (∂²ρ∂X²[p] * generate_Y_matrix_el(1, 1, ∂²ρ∂X²[p],
            ∂²ρ∂XY[p],
            ∂²ρ∂XZ[p],
            ∂²ρ∂Y²[p],
            ∂²ρ∂YZ[p],
            ∂²ρ∂Z²[p]) + ∂²ρ∂XY[p] * generate_Y_matrix_el(2, 1, ∂²ρ∂X²[p],
            ∂²ρ∂XY[p],
            ∂²ρ∂XZ[p],
            ∂²ρ∂Y²[p],
            ∂²ρ∂YZ[p],
            ∂²ρ∂Z²[p]) + ∂²ρ∂XZ[p] * generate_Y_matrix_el(3, 1, ∂²ρ∂X²[p],
            ∂²ρ∂XY[p],
            ∂²ρ∂XZ[p],
            ∂²ρ∂Y²[p],
            ∂²ρ∂YZ[p],
            ∂²ρ∂Z²[p])) (m in 1:3, n in 1:3) grad=false

        @tullio r⃗[p,dim] += -inv_H[dim,1,p] * ∂ρ∂X[p] - inv_H[dim,2,p] * ∂ρ∂Y[p] - inv_H[dim,3,p] * ∂ρ∂Z[p]
    end
    r⃗, transpose(Φ_r.^2) * f.occ_no
end

find_critical_ρ_points (generic function with 1 method)

In [19]:
r⃗_found, ρ = find_critical_ρ_points(r⃗, f)

(Float32[-2.7772922f-18 1.1174581f-16 5.467542f-16; -8.218856f-7 7.153466f-18 0.6703762; … ; -2.7772912f-18 1.1174297f-16 2.1523778f-5; 0.0 12.557076 12.410515], [0.2637181253318364, 0.41073043323513675, 0.4107304399588434, 6.173388290206454e-5, 2.9519711460075918e-5, 0.00011567759997437384, 1.9402665179327345e-5, 0.4107304343738148, 0.26371813522410353, 3.3823507507632224e-16])

In [20]:
r⃗_found

10×3 Matrix{Float32}:
 -2.77729f-18   1.11746f-16   5.46754f-16
 -8.21886f-7    7.15347f-18   0.670376
  9.64199f-19   1.21076f-17  -0.670377
  1.59395f-17   4.45761       2.72123f-16
  3.40471      -3.40471      -9.35475f-17
  2.57315       3.21644       0.532813
  3.18416       3.18416       2.3226
 -1.6956f-18    7.15339f-18   0.670375
 -2.77729f-18   1.11743f-16   2.15238f-5
  0.0          12.5571       12.4105

In [21]:
ρ

10-element Vector{Float64}:
 0.2637181253318364
 0.41073043323513675
 0.4107304399588434
 6.173388290206454e-5
 2.9519711460075918e-5
 0.00011567759997437384
 1.9402665179327345e-5
 0.4107304343738148
 0.26371813522410353
 3.3823507507632224e-16

#### Test with GPU

In [91]:
using BenchmarkTools

f = read_wfn(filepath, device = gpu);
r⃗ = CUDA.rand(10_000,3)

@btime find_critical_ρ_points(r⃗, f)

  4.995 s (10501234 allocations: 322.30 MiB)


(Float32[-0.04006796 -0.025019078 1.2399396; -0.04006796 -0.025019078 1.2399396; … ; -0.040067956 -0.025019074 1.2399393; 2.2730927 -0.032838367 1.3250797], Float32[0.30872256, 0.30872256, 0.30872262, 1.4113541f-16, 0.30872256, 9.767692f-15, 2.0635456f-19, NaN, 0.02077407, NaN  …  0.02077407, 1.8390856f-19, 1.0121274f-14, 0.020774059, 0.02077407, 0.2992517, 0.30872262, 4.530843f-15, 0.30872262, 0.020774059])

In [93]:
r⃗_found, ρ = find_critical_ρ_points(r⃗, f);
r⃗_found

10000×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  -0.040068   -0.0250191     1.23994
  -0.040068   -0.0250191     1.23994
  -0.040068   -0.0250191     1.23994
   0.44936    24.4526        7.23946
  -0.040068   -0.0250191     1.23994
   5.08533   -22.5912        1.90817
   4.77658    17.3459       27.0953
 NaN         NaN           NaN
   2.27309    -0.0328383     1.32508
 NaN         NaN           NaN
   3.61948    22.5728       -9.47456
   1.14725    -0.00723022   -0.696688
  -0.040068   -0.0250191     1.23994
   ⋮                       
   2.27309    -0.0328383     1.32508
  -0.040068   -0.0250191     1.23994
   2.27309    -0.0328383     1.32508
 NaN         NaN           NaN
   4.91356    22.6413       -1.36351
   2.27309    -0.0328384     1.32508
   2.27309    -0.0328383     1.32508
   1.14725    -0.00723022   -0.696688
  -0.040068   -0.0250191     1.23994
   5.28722    22.7659        1.22238
  -0.040068   -0.0250191     1.23994
   2.27309    -0.0328384     1.32508

In [94]:
ρ

10000-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
   0.30872256
   0.30872256
   0.30872262
   7.303408f-23
   0.30872256
   2.1211922f-21
   5.75133f-23
 NaN
   0.02077407
 NaN
   5.7594542f-21
   0.2992517
   0.30872256
   ⋮
   0.020774068
   0.30872256
   0.02077407
 NaN
   2.0414646f-21
   0.020774059
   0.02077407
   0.2992517
   0.30872262
   9.393425f-22
   0.30872262
   0.020774059

#### Test with CPU

In [95]:
f = read_wfn(filepath, device = cpu);
r⃗ = rand(Float32, 10_000, 3)

@btime find_critical_ρ_points(r⃗, f)

  38.260 s (8170 allocations: 71.68 GiB)


(Float32[1.1472485 -0.007230365 -0.6966878; NaN NaN NaN; … ; 2.2730925 -0.03283799 1.3250792; -0.40643695 18.199875 2.4377787], [0.2992517536622811, NaN, 0.0207740855890442, 0.02077408170390762, 0.2950741111159268, 0.30872276364983486, 0.02077409089898389, 0.3087227500873072, 6.152471264373983e-15, 0.30872275005146566  …  2.403474847625511e-13, 2.2986621685320255e-15, 5.589825450885418e-16, 0.020774090898161745, 9.417703438741257e-15, 4.47024548858571e-15, 2.7548219673263433e-14, 1.1100331681083055e-13, 0.020774085589044207, 5.80228745789714e-15])

### Other helper functions

In [96]:
#Calculate ρ for the n points in r⃗
get_electronic_density(r⃗, f)

10000-element Vector{Float64}:
   0.2992517536622811
 NaN
   0.0207740855890442
   0.02077408170390762
   0.2950741111159268
   0.30872276364983486
   0.02077409089898389
   0.3087227500873072
   3.6506818963893095e-15
   0.30872275005146566
   0.02077409089898389
   0.30872275005146566
   9.743082871710655e-17
   ⋮
   0.29925174559637735
   0.30872276364983486
   1.422751213499517e-13
   1.3745147299920554e-15
   3.3449720081520977e-16
   0.020774090898161745
   5.594778829369104e-15
   2.668668504691964e-15
   1.643504744588627e-14
   6.652606478825662e-14
   0.020774085589044207
   3.4770282946194578e-15

In [97]:
#Loss function for gradient descent. Not recommended.
get_sum_squared_gradients(r⃗, f)

NaN

In [ ]:
#Unstable and outdated method using gradient descent. Do not execute
#p, ρ = find_points(r⃗, f, iters=100, η = 0.05)